In [73]:
def Strategy_4():
    import tushare as ts
    import os
    import datetime
    import pymongo
    import pandas

    client = pymongo.MongoClient("127.0.0.1", 27017)
    
    today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    stocks = client.Common.Stock.find({
        "$and": [
            {"value": {"$exists": True}},
            {"value": {"$ne": 0}}
        ]})

    found = []

    for item in stocks:
        code = item["code"]
        collection = client.Trade[code]
        res = collection.find_one({"date": today})
        if not res:
            continue
        if "last_close" not in res:
            continue
        if res["close"] <= res["last_close"]:
            continue
        value = res["tradeValue"] * 10000
        if "amount" in res:
            amount = res["amount"]
        else:
            amount = res["volume"] * (res["open"] + res["close"]) / 2 * 100
        item = {
            "code": code,
            "name": item["name"],
            "value(亿)": value / 1e8,
            "amount(亿)": amount / 1e8,
            "factor": amount / value
        }
        found.append(item)
        
    found = sorted(found, key=lambda item: item["factor"], reverse=True)

    print(pandas.DataFrame(found[:10], columns=["code", "name", "factor", "amount(亿)", "value(亿)"]))

Strategy_4()

     code  name    factor  amount(亿)   value(亿)
0  300634  彩讯股份  0.651501  15.512207  23.809951
1  002864  盘龙药业  0.565316   7.885584  13.948979
2  300504  天邑股份  0.477885  15.139965  31.681163
3  300624  万兴科技  0.419947  10.504566  25.014000
4  300715  凯伦股份  0.411828   4.353898  10.572120
5  300739  明阳电路  0.408891   6.007268  14.691600
6  002930  宏川智慧  0.393257   6.906232  17.561621
7  300730  科创信息  0.389805   4.196161  10.764768
8  002908  德生科技  0.385816   4.810803  12.469160
9  002856  美芝股份  0.369244   2.938925   7.959294


In [25]:
import tushare as ts
import os
import datetime
import pymongo
import pandas

In [26]:
client = pymongo.MongoClient("127.0.0.1", 27017)